In [132]:
import json
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer

nltk.download('wordnet_ic')
brown_ic = wordnet_ic.ic('ic-brown.dat')
from itertools import product

import os
import itertools
import re

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import numpy as np

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/viviancai/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


# Match game with game keywords

In [271]:

# with open('keyphrases_and_keywords.json', 'r', encoding='utf8') as in_json_file:
#     common = json.load(in_json_file)


# with open('keywords_count_dict.json', 'r', encoding='utf8') as in_json_file:
#     word_count = json.load(in_json_file)

with open('movie_keyphrases_and_keywords.json', 'r', encoding='utf8') as in_json_file:
    common = json.load(in_json_file)


with open('movie_keywords_count_dict.json', 'r', encoding='utf8') as in_json_file:
    word_count = json.load(in_json_file)

# with open('.json', 'r', encoding='utf8') as aa:
#     aa = json.load(aa)

In [272]:
def clean_syns(syn):
    syn = syn.lower()
    cleaned = re.sub('[^A-Za-z]+', ' ', syn).strip()
    if cleaned!=syn:
        return []
    split = cleaned.split()
    split = list(filter(lambda x: len(x)>2, split))
    return split

def get_syns(word):
    res = set()
    syns = wn.synsets(word, pos=wn.NOUN)
    syns.extend(wn.synsets(word, pos=wn.ADJ))
    for sense in syns:
        if sense.pos()=='s':
            sense = sense.similar_tos()[0]
#         print(sense.lemma_names())
        cleaned_syns = list(map(clean_syns, sense.lemma_names()))
        cleaned_syns = list(itertools.chain(*cleaned_syns))
        res.update(cleaned_syns)
        
#         # Derived forms
#         derived = [n.name() for l in sense.lemmas() for n in l.derivationally_related_forms()]
#         derived = list(map(clean_syns, derived))
#         derived = list(itertools.chain(*derived))
#         res.update()
    if word in res:
        res.remove(word)
    return list(res)

    
def should_keep(word):
    if word not in word_count.keys():
        return False
    
    (word,pos) = nltk.pos_tag([word])[0]
    if pos[:2]=='NN' or pos[:2]=='JJ':
        return True
    return False

def add_weight(d):
    for word in list(d):
        count = d[word]
        (word,pos) = nltk.pos_tag([word])[0]
        if pos[:2]=='NN':
            d[word] = d[word]*2
            pos = 'n'
        elif pos[:2]=='JJ':
            d[word] = d[word]*1.5
            pos = 'a'
        
        derived_set = set()
        
        syns = wn.synsets(word, pos=pos)
        if pos=='n':
            for sense in syns:
                if sense.pos() == 's':
                    sense = sense.similar_tos()[0]
                derived = [n.name() for l in sense.lemmas() for n in l.derivationally_related_forms()]
                derived = list(map(clean_syns, derived))
                derived = list(itertools.chain(*derived))
                derived = list(filter(lambda x: nltk.pos_tag([x])[0][1][:2]=='JJ', derived))                
                derived_set.update(derived)

            for derived in derived_set:
                if derived not in d.keys():
                    d[derived] = d[word]*0.9
    return d

def process_info(info):
    lst = list(info['keywords'])
    lst = list(filter(should_keep, lst))
    
    ctr = add_weight(dict(Counter(lst)))
    
    numbers = np.array(list(ctr.values()))
    squared = numbers ** 2
    norm = np.sum(squared)
    norm = np.sqrt(norm)
#     numbers = ctr.values()['vector'].values()
#     print(numbers)
#     print(ctr)
    return ctr, norm
    
    
# keyword -> list of keywords and their synonyms
# keyphrase -> break down to words and for each word, check if its synonym in word_to_synphrase

In [273]:
info_path = 'movie_info/'

# appid_to_vec = dict()
link_to_vec = dict()
count = 0
for filename in os.listdir(info_path):
    if filename.endswith('.json'):
        with open(info_path+filename, 'r', encoding='utf8') as in_json_file:
            info = json.load(in_json_file)
            link = info['link']
            vector, norm = process_info(info)
            link_to_vec[link] = {'link':link, 'vector':vector, 'norm':norm}
    
    count+=1
    if count%100==0:
        print(count)
    if count == 1:
        print(link_to_vec)


{'flakes': {'link': 'flakes', 'vector': {'display': 2, 'bowl': 2, 'cable': 2, 'cool': 2, 'charm': 2, 'bargain': 2, 'comedy': 2, 'corn': 2, 'capitalism': 2, 'breezy': 2, 'dvd': 2, 'look': 2, 'matinee': 2, 'comical': 1.8, 'comic': 1.8, 'capitalistic': 1.8}, 'norm': 7.856207736560942}}
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13

In [274]:

# output_path = 'appid_to_vec.json'
# with open(output_path, 'w') as f:
#     f.write(json.dumps(appid_to_vec, indent=4)+'\n')

output_path = 'movielink_to_vec.json'
with open(output_path, 'w') as f:
    f.write(json.dumps(link_to_vec, indent=4)+'\n')

with open('phrase_word_to_synphrase.json', 'r', encoding='utf8') as in_json_file:
    word_to_synphrase = json.load(in_json_file)

with open('inverse_keyword_phrases.json', 'r', encoding='utf8') as in_json_file:
    inverse = json.load(in_json_file)



In [269]:
# input_tags = ['adventure','great story','graphic','open world','soundtrack']
# input_tags = ['romance','love story','graphic','open world','soundtrack']
input_tags = ['game', 'world-war', 'geralt canon', 'open world', 'friends']

def clean_input(tags):
    lemmatizer = WordNetLemmatizer()
    def helper(tag):
        tag = tag.lower()
        tag = re.sub('[^A-Za-z]+', ' ', tag).strip()
        if len(tag.split())==1:
            (tag,pos) = nltk.pos_tag([tag])[0]
            if pos == 'n' or pos == 'a':
                tag = lemmatizer.lemmatize(tag, pos)
            else:
                tag = lemmatizer.lemmatize(tag, 'n')
        return tag
    tags = list(map(helper, tags))
    
    stopwords = ['game','play','video']
    tags = list(filter(lambda x:x not in stopwords, tags))
    return tags
    

def process_input_match_phrase(tags):
    res = dict()
    
    phrases = list(filter(lambda x: len(x.split())>1, tags))
    
    words = list(map(lambda x: x.split(), input_tags))
    words = list(itertools.chain(*words))
    
    word_to_tag = dict()
    for word in words:
        for tag in tags:
            if word in tag:
                t = word_to_tag.get(word, list())
                t.append(tag)
                word_to_tag[word] = t
    
    for phrase in phrases:
        if phrase in common['keyphrases']:
            res[phrase] = 3
    
    phrases.remove(phrase)
    
    for word in words:
        (word,pos) = nltk.pos_tag([word])[0]
        if pos=='n':
            synphrases = word_to_synphrase.get(word, list())
            for synphrase in synphrases:
                res[word_to_tag[word]] = 1.5
    
    game_to_score = dict()
    game_to_phrases = dict()
    for k in res.keys():
        games = inverse.get(k, list())
        for game in games:
            score = game_to_score.get(game, 0)
            score += res[k]
            phrases = game_to_phrases.get(game, set())
            phrases.add(k)
            game_to_score[game] = score
            game_to_phrases[game] = phrases
    
    final = {game: (game_to_score[game], game_to_phrases[game]) for game in game_to_score.keys()}
    final = [(game, score_phrases) for (game, score_phrases) in final.items()]
    final = sorted(final, key=lambda x:(-x[1][0], -len(x[1][1]), x[0]))
    return final

input_tags = clean_input(input_tags)
print(process_input_match_phrase(input_tags))
    
    
    
def process_input(tags):
    tags = list(map(lambda x: x.split(), input_tags))
    tags = list(itertools.chain(*tags))
    
    # Get syns for each tag
    tag_to_syns = dict()
    for tag in tags:
        syns = tag_to_syns.get(tag, list())
        syns.extend(get_syns(tag))
        tag_to_syns[tag] = syns
    
    # Get all the words to put in the final vector
    allwords = list(tags)
    for syns in tag_to_syns.values():
        allwords.extend(syns)
    
    ctr = Counter(allwords)
    ctr = dict(ctr)
    
    # Adjust weight
    for word in set(tags):
        count = ctr[word]
        (word,pos) = nltk.pos_tag([word])[0]
        if pos[:2]=='NN':
            if word in tags:
                ctr[word] = ctr[word]*2
            else:
                ctr[word] = ctr[word]*1.75
            pos = 'n'
        elif pos[:2]=='JJ':
            if word in tags:
                ctr[word] = ctr[word]*1.75
            else:
                ctr[word] = ctr[word]*1.5
            pos = 'a'
        else:
            continue
        
        derived_set = set()
        
        syns = wn.synsets(word, pos=pos)
        if pos=='n':
            for sense in syns:
                derived = [n.name() for l in sense.lemmas() for n in l.derivationally_related_forms()]
                derived = list(map(clean_syns, derived))
                derived = list(itertools.chain(*derived))
                derived = list(filter(lambda x: nltk.pos_tag([x])[0][1][:2]=='JJ', derived))                
                derived_set.update(derived)

            for derived in derived_set:
                if derived not in ctr.keys():
                    ctr[derived] = ctr[word]*0.9
    
    numbers = np.array(list(ctr.values()))
    squared = numbers ** 2
    norm = np.sum(squared)
    norm = np.sqrt(norm)
    
    return ctr, norm
    
input_vector, norm = process_input(input_tags)


# lemmatizer = WordNetLemmatizer()
# lemmatizer.lemmatize("romantic")

# stemmer  = SnowballStemmer('english')
# stemmer.stem('romantic')

# stemmer = PorterStemmer()
# stemmer.stem('romance')

[('226700', (6, {'open world', 'world war'})), ('312660', (6, {'open world', 'world war'})), ('378610', (6, {'open world', 'world war'})), ('50130', (6, {'open world', 'world war'})), ('10090', (3, {'world war'})), ('10150', (3, {'open world'})), ('102500', (3, {'open world'})), ('1030830', (3, {'open world'})), ('1030840', (3, {'open world'})), ('1055540', (3, {'open world'})), ('105600', (3, {'open world'})), ('1056640', (3, {'open world'})), ('1056960', (3, {'open world'})), ('107410', (3, {'open world'})), ('1085660', (3, {'open world'})), ('108710', (3, {'open world'})), ('108800', (3, {'open world'})), ('1097840', (3, {'open world'})), ('110800', (3, {'open world'})), ('1116960', (3, {'open world'})), ('1121560', (3, {'open world'})), ('1128000', (3, {'open world'})), ('113400', (3, {'open world'})), ('11450', (3, {'open world'})), ('1151340', (3, {'open world'})), ('1151640', (3, {'open world'})), ('115320', (3, {'open world'})), ('1172380', (3, {'open world'})), ('1172620', (3,

In [267]:
def cos_sim(d1, d2, norm1, norm2):
    intersect = set(d1.keys()).intersection(set(d2.keys()))
    score = sum(d1[k]*d2[k] for k in intersect)
    return (score, intersect)

# cos_sim(input_vector, appid_to_vec['47810']['vector'])
# print(input_vector)
# print(appid_to_vec['47810']['vector'])

res = list(map(lambda x: (x['app_id'], cos_sim(input_vector, x['vector'], norm, x['norm'])), appid_to_vec.values()))
res = sorted(res, key=lambda x:(-x[1][0], x[0]))
res

[('24980',
  (25.665,
   {'account',
    'active',
    'graphic',
    'level',
    'romance',
    'romantic',
    'soundtrack',
    'story',
    'universe'})),
 ('901242',
  (25.665,
   {'account',
    'active',
    'graphic',
    'level',
    'romance',
    'romantic',
    'soundtrack',
    'story',
    'universe'})),
 ('17460',
  (24.665,
   {'active',
    'available',
    'graphic',
    'history',
    'level',
    'narrative',
    'romance',
    'romantic',
    'story',
    'universe'})),
 ('204450',
  (23.925,
   {'active',
    'graphic',
    'history',
    'level',
    'narration',
    'narrative',
    'reality',
    'soundtrack',
    'story',
    'tale'})),
 ('238010',
  (23.665,
   {'active',
    'earth',
    'global',
    'graphic',
    'level',
    'mankind',
    'soundtrack',
    'story',
    'universe'})),
 ('278080',
  (23.665,
   {'active',
    'graphic',
    'history',
    'level',
    'romance',
    'romantic',
    'soundtrack',
    'story'})),
 ('260230',
  (23.165,
   

# Match keyphrase directly

In [207]:
with open('phrase_word_to_synphrase.json', 'r', encoding='utf8') as in_json_file:
    word_to_synphrase = json.load(in_json_file)


In [211]:
# input_tags = ['adventure','great story','graphic','open world','soundtrack']
# input_tags = ['romance','love story','graphic','open world','soundtrack']
input_tags = ['game', 'world war', 'great story', 'open world']

input_phrase = list(filter(lambda x: len(x.split())>1, input_tags))

input_phrase

['world war', 'great story']